In [2]:
import numpy as np;
import pandas as pd;

from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import RepeatedStratifiedKFold
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import matthews_corrcoef,make_scorer
mccScorer = make_scorer(matthews_corrcoef)
from numpy import mean
from numpy import std

from google.colab import drive
drive.mount("/content/gdrive")

from sklearn.pipeline import Pipeline
from sklearn.model_selection import cross_validate

from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import f_classif


Mounted at /content/gdrive


In [3]:
#read training dataset 1
data1 = pd.read_csv('/content/gdrive/My Drive/Colab Notebooks/Dataset_1_Training.csv')

X_train_actual_1=data1[:-2].T[1:].values

y_train_actual_1 = data1[-2:].T[-130:].T.values

In [4]:
#read training dataset 2
data2 = pd.read_csv('/content/gdrive/My Drive/Colab Notebooks/Dataset_2_Training.csv')

X_train_actual_2=data2[:-4].T[1:].values

y_train_actual_2 = data2[-4:].T[-340:].T.values

In [5]:
co1={'X':X_train_actual_1,'Y': y_train_actual_1[0]};
co2={'X':X_train_actual_1,'Y': y_train_actual_1[1]};

co3={'X':X_train_actual_2,'Y': y_train_actual_2[0]};
co4={'X':X_train_actual_2,'Y': y_train_actual_2[1]};
co5={'X':X_train_actual_2,'Y': y_train_actual_2[2]};
co6={'X':X_train_actual_2,'Y': y_train_actual_2[3]};

In [1]:
def feature_selector(X,Y,numberOfFeaturesToBeSelected=1):
  if(numberOfFeaturesToBeSelected == 1):
    totalNumberOfFeatures=X.shape[1]
    numberOfFeaturesToBeSelected=totalNumberOfFeatures//2;
  # define feature selection
  fs = SelectKBest(score_func=f_classif, k=numberOfFeaturesToBeSelected)
  # apply feature selection
  return fs.fit(X, Y)

In [11]:
def RandomForest_with_cv(X,Y):
  # define the model
  model = RandomForestClassifier()
  
  # evaluate the model
  cv = RepeatedStratifiedKFold(n_splits=10, n_repeats=3, random_state=1)
  
  scoringReports={'mccScorer': mccScorer,'accuracy': 'accuracy','precision':'precision','recall':'recall', 'f1':'f1'}
  
  X_selected_features=feature_selector(X,Y,numberOfFeaturesToBeSelected=1).transform(X);
  scores = cross_validate(model,X_selected_features , Y.astype('int'), scoring=scoringReports, cv=cv)

  # summarize result
  print('MCC Score- Mean: %.3f SD: (%.3f)' % (mean(scores['test_mccScorer']), std(scores['test_mccScorer'])))
  print(list(scores.keys()))
  print("Means",mean(list(scores.values()),axis=1))
  print("SD",std(list(scores.values()),axis=1))

In [ ]:
def train_RandForest():
  print("RF-CO1 started")
  RandomForest_with_cv(**co1);
  print("RF-CO2 started")
  RandomForest_with_cv(**co2);
  print("RF-CO3 started")
  RandomForest_with_cv(**co3);
  print("RF-CO4 started")
  RandomForest_with_cv(**co4);
  print("RF-CO5 started")
  RandomForest_with_cv(**co5);
  print("RF-CO6 started")
  RandomForest_with_cv(**co6)

In [10]:
RandomForest_with_cv(**co1);

/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:900: RuntimeWarning: invalid value encountered in double_scalars
  mcc = cov_ytyp / np.sqrt(cov_ytyt * cov_ypyp)
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:900: RuntimeWarning: invalid value encountered in double_scalars
  mcc = cov_ytyp / np.sqrt(cov_ytyt * cov_ypyp)
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn

MCC Score- Mean: 0.339 SD: (0.287)
['fit_time', 'score_time', 'test_mccScorer', 'test_accuracy', 'test_precision', 'test_recall', 'test_f1']
Means [2.81682271 0.05759319 0.33908209 0.78717949 0.61111111 0.31111111
 0.38793651]
SD [0.06447283 0.00628119 0.28720623 0.08112457 0.40673345 0.23953401
 0.26108887]


In [ ]:
def RandomForest_feat_hyp_tuning(X,Y):
  pipeline = Pipeline(
    [
      ('selector',SelectKBest(f_classif)),
      ('model',RandomForestClassifier(class_weight='balanced',random_state = 1))
    ]
  )
  search = GridSearchCV(estimator = pipeline,param_grid = {
  'selector__k':[int(x) for x in np.linspace(start = 5 , stop = X.shape[1], num = 40)] , 
  'model__n_estimators':[int(x) for x in np.linspace(start = 200, stop = 2000, num = 10)]},
    n_jobs=-1,
    scoring=mccScorer,
    cv=2,
    verbose=2)
  
  result = search.fit(X,Y.astype('int'))
  # summarize result
  print('Best MCC Score: %s' % result.best_score_)
  print('Best Hyperparameters: %s' % result.best_params_)

In [ ]:
RandomForest_feat_hyp_tuning(**co3);

In [ ]:
RandomForest_feat_hyp_tuning(**co4);

Fitting 2 folds for each of 400 candidates, totalling 800 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 2 concurrent workers.
[Parallel(n_jobs=-1)]: Done  37 tasks      | elapsed:  2.5min
[Parallel(n_jobs=-1)]: Done 158 tasks      | elapsed: 12.7min
[Parallel(n_jobs=-1)]: Done 361 tasks      | elapsed: 34.5min
[Parallel(n_jobs=-1)]: Done 644 tasks      | elapsed: 77.0min
[Parallel(n_jobs=-1)]: Done 800 out of 800 | elapsed: 106.4min finished


Best MCC Score: 0.05436602822195624
Best Hyperparameters: {'model__n_estimators': 200, 'selector__k': 1406}


In [ ]:
RandomForest_feat_hyp_tuning(**co5);

Fitting 2 folds for each of 400 candidates, totalling 800 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 2 concurrent workers.
[Parallel(n_jobs=-1)]: Done  37 tasks      | elapsed:  2.6min
[Parallel(n_jobs=-1)]: Done 158 tasks      | elapsed: 13.3min
[Parallel(n_jobs=-1)]: Done 361 tasks      | elapsed: 37.8min
[Parallel(n_jobs=-1)]: Done 644 tasks      | elapsed: 88.6min
[Parallel(n_jobs=-1)]: Done 800 out of 800 | elapsed: 124.5min finished


Best MCC Score: 0.7897798096617333
Best Hyperparameters: {'model__n_estimators': 1000, 'selector__k': 5}


In [ ]:
RandomForest_feat_hyp_tuning(**co6);

Fitting 2 folds for each of 400 candidates, totalling 800 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 2 concurrent workers.
[Parallel(n_jobs=-1)]: Done  37 tasks      | elapsed:  2.6min
[Parallel(n_jobs=-1)]: Done 158 tasks      | elapsed: 13.6min
[Parallel(n_jobs=-1)]: Done 361 tasks      | elapsed: 39.1min
[Parallel(n_jobs=-1)]: Done 644 tasks      | elapsed: 92.5min
[Parallel(n_jobs=-1)]: Done 800 out of 800 | elapsed: 131.0min finished


Best MCC Score: 0.00020529166551975196
Best Hyperparameters: {'model__n_estimators': 200, 'selector__k': 18228}
